In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd 
import os 

## we load the csv file and we add a column for the label diagnosis. 

cc_meta = pd.read_csv('/content/drive/My Drive/ADReSS/train/cc_meta_data.txt', sep=';')
cc_meta['label'] = [0]*len(cc_meta)
cd_meta = pd.read_csv('/content/drive/My Drive/ADReSS/train/cd_meta_data.txt', sep=';') 
cd_meta['label'] = [1]*len(cd_meta)
cc_meta.head()
list(cc_meta)



['ID   ', ' age', ' gender ', 'mmse', 'label']

In [3]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import re 
from nltk.tokenize import word_tokenize

def convert_list_to_string(list_strings):
    return_string = ""
    for string in list_strings:

        string = string.replace("*PAR:","")
        string = string.replace("%mor:","")
        string = string.replace("%gra:","")
        string = string.replace("\t","").replace("\n"," ")
        string = string.replace("~"," ")

        string = re.sub(r'[\d]+', '', string)
        #string = string.replace('[', '')
        #string = string.replace(']', '')
        #string = string.replace('(', '')
        #string = string.replace(')', '')
        string = string.replace('exc', '')

        return_string += string
    return return_string

def convert_mor_to_list(list_string):
    return_list = []
    element_list = list_string.split(" ")
    for element in element_list:
        if "|" in element:
            return_list.append(element.split("|")[0])
    return return_list

def convert_gra_to_list(list_string):
    return_list = []
    element_list = list_string.split(" ")
    for element in element_list:
        if "|" in element:
            return_list.append(element.split("|")[2])
    return return_list

def file_pos_extraction(fullpath):
  with open(fullpath, 'r') as input_file:
    id_string = input_file.name.split('/')[-1]
    #print(id_string)
    result = re.search('(.*).cha',id_string)
    id = result.group(1)

    par_output = []
    mor_output = []
    gra_output = []

    i = 0
    list_files = list(input_file)
    while i < len(list_files):
        par_list = []
        mor_list = []
        gra_list = []
        if i == 5:
            if "_X_" in list_files[i] or "|Control|" in list_files[i] or "|ProbableAD|" in list_files[i] or "|PossibleAD|" in list_files[i] :
                pass
            else:
                return [],[],[], False
        if "*PAR:" in list_files[i]:
            #print(list_files[i])
            while "%mor" not in list_files[i]:
                par_list.append(list_files[i])
                i += 1
                if "@End" in list_files[i]:
                    break
            if "@End" in list_files[i]:
                break

            if "%mor" in list_files[i]:
                #print(list_files[i])
                while "%gra" not in list_files[i]:
                    mor_list.append(list_files[i])
                    i += 1

                if "%gra" in list_files[i]:
                    #print(list_files[i])
                    while "*PAR" not in list_files[i] and "*INV" not in list_files[i] and "@End" not in list_files[i]:
                        gra_list.append(list_files[i])
                        i += 1
        else:
            i += 1
        w = word_tokenize(convert_list_to_string(par_list))
        if '\x15_\x15' in w:
            w.remove('\x15_\x15')
        m = convert_mor_to_list(convert_list_to_string(mor_list))
        g = convert_gra_to_list(convert_list_to_string(gra_list))

        if len(w) > 0:
            par_output += w
        if len(m) > 0:
            mor_output.append(m)
        if len(g) > 0:
            gra_output.append(g)
    #return par_output,mor_output,gra_output,id
    return par_output

In [5]:
cc_meta['text'] = [None] * len(cc_meta)

for index, row in cc_meta.iterrows():
  str_id = row['ID   '].replace(' ','')
  file_path = '/content/drive/My Drive/ADReSS/train/transcription/Control/' + str_id + '.cha'

  if (cc_meta.at[index,'text'] == None):
    output = file_pos_extraction(file_path)
    cc_meta.at[index,'text'] = ' '.join(output)

cc_meta.head()

,ID,age,gender,mmse,label,text
0,S001,74,male,NA,0,well there 's a mother standing there & uh & u...
1,S002,62,female,30,0,somebody 's getting cookies out_of the cookie ...
2,S003,69,female,29,0,okay . [ + ] there 's a little boy and < he 's...
3,S004,71,female,30,0,are you ready ? [ + ] & um well the sink is ov...
4,S005,74,female,30,0,okay . [ + ] < many & dish or > [ // ] the mot...


In [6]:
cd_meta['text'] = [None] * len(cd_meta)

for index, row in cd_meta.iterrows():
  str_id = row['ID   '].replace(' ','')
  file_path = '/content/drive/My Drive/ADReSS/train/transcription/Dementia/' + str_id + '.cha'

  if (cd_meta.at[index,'text'] == None):
    output = file_pos_extraction(file_path)
    try:
      cd_meta.at[index,'text'] = ' '.join(output)
    except: 
      cd_meta.at[index,'text'] = None 
cd_meta.head()

,ID,age,gender,mmse,label,text
0,S079,59,male,11,1,mhm . there 's a young boy & uh going in a coo...
1,S080,55,male,19,1,okay he 's fallin ( g ) off a chair [ : stool ...
2,S081,69,male,19,1,< well the kid > [ // ] the girl 's laughin ( ...
3,S082,66,male,11,1,& =clears : throat well & =clears : throat & u...
4,S083,52,male,30,1,None


In [7]:
final_df = pd.concat([cd_meta, cc_meta], ignore_index=True)
final_df.head(100)
print(final_df)

import pickle 
with open('/content/drive/My Drive/ADReSS/train/transcriptions.pickle', 'wb') as f:
    pickle.dump(final_df, f)

     ID      age  ... label                                               text
0    S079     59  ...     1  mhm . there 's a young boy & uh going in a coo...
1    S080     55  ...     1  okay he 's fallin ( g ) off a chair [ : stool ...
2    S081     69  ...     1  < well the kid > [ // ] the girl 's laughin ( ...
3    S082     66  ...     1  & =clears : throat well & =clears : throat & u...
4    S083     52  ...     1                                               None
..     ...   ...  ...   ...                                                ...
103  S071     74  ...     0  the kids are in the cookies . the stool is fal...
104  S072     75  ...     0  you mean right now tell you ? [ + ] & uh the b...
105  S073     61  ...     0  well ‡ the boy 's trying to get in this cookie...
106  S076     78  ...     0  mhm . [ + ] mhm a_lot_of things are happening ...
107  S077     77  ...     0  alright . [ + ] the little boy [ // ] girl 's ...

[108 rows x 6 columns]


In [8]:
test = pd.read_csv('/content/drive/My Drive/ADReSS/test/meta_data_inferred.txt', sep=';',names=["ID", "age", "gender", "mmse","label"]) 
test.head()

test['text'] = [None] * len(test)



for index, row in test.iterrows():
  str_id = row['ID'].replace(' ','')
  file_path = '/content/drive/My Drive/ADReSS/test/transcription/' + str_id + '.cha'


  if (test.at[index,'label'] == None):
    if mmse > 24: 
      test.at[index,'label'] = 0 
    else: 
      test.at[index,'label'] = 1
  elif (test.at[index,'label'] == 'Control'):
      test.at[index,'label'] = 0
  elif (test.at[index,'label'] == 'Dementia'):
      test.at[index,'label'] = 1

  if (test.at[index,'text'] == None):
    output = file_pos_extraction(file_path)
    print(output)
    test.at[index,'text'] = ' '.join(output)

['&', 'uh', 'this', 'boy', 'is', 'about', 'to', 'fall', 'off', 'o', '(', 'f', ')', 'the', 'stool', '.', 'the', 'mother', 'is', '&', 'uh', 'washing', 'dishes', 'and', 'the', 'water', "'s", 'spilling', 'over', 'on', 'the', '&', 'uh', '(', '.', ')', 'kitchen', 'floor', '.', 'the', 'wind', 'is', 'blowing', 'the', 'curtains', '.', 'the', 'little', 'girl', 'is', 'laughing', 'at', 'her', 'brother', 'who', "'s", 'taking', 'a', 'cookie', 'out_of', 'the', 'cookie', 'jar', '.', 'think', 'that', "'s", 'it', '.', '[', '+', ']']
['&', 'uh', 'a', 'boy', 'with', 'a', 'cookie', 'in', 'his', 'one', 'hand', 'and', 'his', 'hand', 'in', 'the', 'cookie', 'jar', '.', '[', '+', 'gram', ']', 'standing', 'on', 'a', 'stool', 'which', 'is', 'tipping', 'over', '.', '[', '+', 'gram', ']', 'the', 'little', 'girl', "'s", 'got', 'her', 'hand', 'up', 'for', 'one', '.', 'and', 'I', 'do', "n't", 'know', 'what', 'the', 'hand', 'means', 'to', 'the', 'mouth', '.', '[', '+', ']', 'does', 'it', 'mean', 'she', 'wants', 'to', '

In [0]:
test.head()

with open('/content/drive/My Drive/ADReSS/test/transcriptions.pickle', 'wb') as f:
    pickle.dump(test, f)